Luana Rauber Sostisso, Matheus Azevedo e Gabriel Ottonelli

## Trabalho 1 - Wikipedia Pessoas

In [4]:
import os
import re
import requests
import time
import threading
from bs4 import BeautifulSoup
from queue import Queue
from queue import Empty as QueueEmpty
from urllib.parse import urljoin, unquote
import shutil

In [5]:
# --- Configurações Iniciais ---
URL_BASE = "https://pt.wikipedia.org"
URL_INICIAL = urljoin(URL_BASE, "/wiki/Wikipédia:Página_principal")
HEADERS = {"User-Agent": "Mozilla/5.0"}
PASTA_OUTPUT = "LINKS_PESSOAS"
TARGET_COUNT = 1000
NUM_THREADS = 10 # para o uso em versao paralela

# --- Estruturas de Dados ---
fila_prioritaria = Queue()             # lista de links que foram extraidos de paginas que pertenciam a pessoas
fila_normal = Queue()                  # lista de links extraidos de paginas que nao sao de pessoas
visitados = set()                      # lista de links visitados, nao visitamos links repetidos
links_pessoas_encontrados = []         # lista de links que foram extraidos de paginas que pertenciam a pessoas
data_lock = threading.Lock()           # estrutura de lock do python

In [6]:
# --- Funcoes ---
def eh_pagina_de_pessoa(soup):
    infobox = soup.find("table", {"class": "infobox_v2"})
    if not infobox:
      infobox = soup.find("table", {"class": "infobox"})
    if not infobox:
      infobox = soup.find("table", {"class": "infobox infobox_v2"})

    if infobox:
        cabecalhos = infobox.find_all("th")
        for th in cabecalhos:
            if "Nascimento" in th.get_text():
                return True
    return False

def extrair_links_validos(soup):
    """Extrai todos os links internos de artigos da Wikipédia de uma página."""
    links_encontrados = set()
    for link in soup.find_all("a", href=re.compile(r"^(/wiki/)((?!:).)*$")):
        href = link.get("href")
        if href:
            url_completa = urljoin(URL_BASE, href)
            links_encontrados.add(url_completa)
    return list(links_encontrados)

def salvar_pagina_pessoa(url_pessoa, conteudo_html):
    """
    Salva o conteúdo HTML de uma página de pessoa em um arquivo.
    """
    try:
        nome_base = url_pessoa.split('/wiki/')[-1] # Extrai o nome do link, no padrao Wiki eh a parte final do link
        caminho_completo = os.path.join(PASTA_OUTPUT, (nome_base).strip() + '.html')

        with open(caminho_completo, "wb") as arquivo: # wb (write bytes) pq resposta.content é uma sequência de bytes
            arquivo.write(conteudo_html)

        print(f"   --> Arquivo salvo: {caminho_completo}")

    except Exception as e:
        print(f"   --> ERRO ao salvar o arquivo para a URL {url_pessoa}: {e}")

In [7]:
# --- Criacao de pasta para salvar, se a pasta tem coisas -> limpa ---
if os.path.exists(PASTA_OUTPUT):
    shutil.rmtree(PASTA_OUTPUT)

os.makedirs(PASTA_OUTPUT, exist_ok=True)
print(f"Pasta '{PASTA_OUTPUT}' criada")

Pasta 'LINKS_PESSOAS' criada


In [8]:
print(f"Iniciando busca a partir de: {URL_INICIAL}")

# a partir da pagina inicial popular a lista fila_normal
try:
    resposta = requests.get(URL_INICIAL, headers=HEADERS, timeout=10)
    soup = BeautifulSoup(resposta.content, "html.parser")
    links_iniciais = extrair_links_validos(soup)
    for link in links_iniciais:
        if link not in visitados:
            fila_normal.put(link)
            visitados.add(link)
    print(f"Encontrados {len(links_iniciais)} links na página principal")
except requests.RequestException as e:
    print(f"Erro ao acessar a página inicial: {e}.")


Iniciando busca a partir de: https://pt.wikipedia.org/wiki/Wikipédia:Página_principal
Encontrados 116 links na página principal


##### Criamos duas versoes do trabalho, uma em paralelo usando threads e outra sequencial. Ambas fazem a mesma funcao, mas como o numero de pessoas a serem extraidas era grande, tivemos que usar a versao paralela para coletar os links

## Versao Paralela usando Threads


In [9]:
"""Função que cada thread executará para processar as páginas."""
def worker():
    global visitados
    global links_pessoas_encontrados
    global visited_count

    while True:
        with data_lock:
            if len(links_pessoas_encontrados) >= TARGET_COUNT:
                break
        try:
            pagina_atual = fila_prioritaria.get_nowait()
            fonte = "PRIORITÁRIA"
        except QueueEmpty:
            try:
                pagina_atual = fila_normal.get_nowait()
                fonte = "NORMAL"
            except QueueEmpty:
                time.sleep(0.3)
                # Se a execução chegou a este ponto, significa que tanto a fila prioritária quanto a normal
                # estão vazias. Em vez de deixar a thread em um loop infinito (busy-waiting),
                # o que consumiria recursos de CPU desnecessariamente, fazemos uma pequena pausa.
                #
                # Esta pausa de 0.3 segundos é uma aposta de que, nesse meio tempo, outras threads
                # ativas encontrarão e adicionarão novos links às filas.
                #
                # Embora não seja a solução mais otimizada (que seria usar 'fila.get()' bloqueante),
                # ela é funcional, simples de entender e adequada para a escala deste crawler.
                continue

        with data_lock:
            visited_count += 1

        print(f"[{threading.current_thread().name}] Visitando ({fonte}): {pagina_atual}")

        try:
            resposta = requests.get(pagina_atual, headers=HEADERS, timeout=10)
            conteudo_pagina = resposta.content
            soup_pagina = BeautifulSoup(conteudo_pagina, "html.parser")

            if eh_pagina_de_pessoa(soup_pagina):
                print(f"PESSOA ENCONTRADA! -> {pagina_atual} por [{threading.current_thread().name}]")

                with data_lock:
                    if len(links_pessoas_encontrados) < TARGET_COUNT:
                        links_pessoas_encontrados.append(pagina_atual)
                        count = len(links_pessoas_encontrados)
                        print(f"---------- Progresso: {count}/{TARGET_COUNT} ----------")
                    else:
                        continue

                salvar_pagina_pessoa(pagina_atual, conteudo_html=conteudo_pagina)

                novos_links = extrair_links_validos(soup_pagina)
                for link in novos_links:
                    with data_lock:
                        if link not in visitados:
                            visitados.add(link)
                            fila_prioritaria.put(link)
            else:
                novos_links = extrair_links_validos(soup_pagina)
                for link in novos_links:
                    with data_lock:
                        if link not in visitados:
                            visitados.add(link)
                            fila_normal.put(link)
            time.sleep(0.1)

        except requests.RequestException as e:
            print(f"  -> [{threading.current_thread().name}] Erro de request em {pagina_atual}: {e}")
        except Exception as e:
            print(f"  -> [{threading.current_thread().name}] Erro inesperado em {pagina_atual}: {e}")

In [10]:
# Trecho para executar versao paralela, se for executar sequencial, nao rode
visited_count = 0

threads = []
for i in range(NUM_THREADS):
    thread = threading.Thread(target=worker, name=f"Thread-{i+1}")
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

proportion = len(links_pessoas_encontrados) / visited_count if visited_count > 0 else 0
print(f"\n--- Versão Paralela ---")
print(f"Total de páginas de pessoas encontradas: {len(links_pessoas_encontrados)}")
print(f"Total de páginas visitadas: {visited_count}")
print(f"Proporção de páginas coletadas / páginas visitadas: {proportion:.4f}")

Streaming output truncated to the last 5000 lines.
   --> Arquivo salvo: LINKS_PESSOAS/Zhang_Wentian.html
[Thread-5] Visitando (PRIORITÁRIA): https://pt.wikipedia.org/wiki/Zhang_Xueliang
[Thread-9] Visitando (PRIORITÁRIA): https://pt.wikipedia.org/wiki/Anti-imperialista
[Thread-10] Visitando (PRIORITÁRIA): https://pt.wikipedia.org/wiki/Margem_da_%C3%81gua
[Thread-4] Visitando (PRIORITÁRIA): https://pt.wikipedia.org/wiki/Wang_Fuzhi
[Thread-3] Visitando (PRIORITÁRIA): https://pt.wikipedia.org/wiki/Qufu[Thread-6] Visitando (PRIORITÁRIA): https://pt.wikipedia.org/wiki/Jonathan_Spence

PESSOA ENCONTRADA! -> https://pt.wikipedia.org/wiki/Nikita_Kruchov por [Thread-8]
---------- Progresso: 455/1000 ----------
   --> Arquivo salvo: LINKS_PESSOAS/Nikita_Kruchov.html
[Thread-9] Visitando (PRIORITÁRIA): https://pt.wikipedia.org/wiki/Segunda_Frente_Unida
[Thread-1] Visitando (PRIORITÁRIA): https://pt.wikipedia.org/wiki/O_Esp%C3%ADrito_das_Leis
PESSOA ENCONTRADA! -> https://pt.wikipedia.org/wiki/Zh

## Versao sequencial

In [ ]:
visited_count = 0

while len(links_pessoas_encontrados) < TARGET_COUNT:
    if fila_prioritaria.empty() and fila_normal.empty():
        print("As filas de links estao vazias. Encerrando a busca.")
        break

    if not fila_prioritaria.empty():
        pagina_atual = fila_prioritaria.get()
        fonte = "PRIORITÁRIA"
    else:
        pagina_atual = fila_normal.get()
        fonte = "NORMAL"

    visited_count += 1

    print(f"[{len(links_pessoas_encontrados)}/{TARGET_COUNT}] Visitando ({fonte}): {pagina_atual}")

    try:
        resposta = requests.get(pagina_atual, headers=HEADERS, timeout=10)
        conteudo_pagina = resposta.content
        soup_pagina = BeautifulSoup(resposta.content, "html.parser")

        if eh_pagina_de_pessoa(soup_pagina):
            print(f"PESSOA ENCONTRADA! -> {pagina_atual}")
            links_pessoas_encontrados.append(pagina_atual)
            salvar_pagina_pessoa(pagina_atual, conteudo_html=conteudo_pagina)
            novos_links = extrair_links_validos(soup_pagina)
            for link in novos_links:
                if link not in visitados:
                    fila_prioritaria.put(link)
                    visitados.add(link)
        else:
            novos_links = extrair_links_validos(soup_pagina)
            for link in novos_links:
                if link not in visitados:
                    fila_normal.put(link)
                    visitados.add(link)
        time.sleep(0.1)
    except requests.RequestException as e:
        print(f"  -> Erro ao acessar {pagina_atual}: {e}")
        continue
    except Exception as e:
        print(f"  -> Ocorreu um erro inesperado ao processar {pagina_atual}: {e}")
        continue


print(f"\n--- Versão Sequencial ---")
print(f"Total de pessoas encontradas: {len(links_pessoas_encontrados)}")
print(f"Total de páginas visitadas: {visited_count}")
proportion = len(links_pessoas_encontrados) / visited_count if visited_count > 0 else 0
print(f"Proporção de páginas coletadas / páginas visitadas: {proportion:.4f}")

### Baixar arquivos do colab

In [11]:
from google.colab import files
import shutil

shutil.make_archive("LINKS_PESSOAS", "zip", "/content/LINKS_PESSOAS")
files.download("LINKS_PESSOAS.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>